In [1]:
import os
from pathlib import Path
import numpy as np
from tqdm.auto import tqdm
import pickle
from collections import defaultdict
import importlib

from themachinethatgoesping.echosounders import kongsbergall, simradraw, index_functions

In [2]:
import themachinethatgoesping as tmtgp
tmtgp.version()

themachinethatgoesping
- version:       0.11.0

modules:
- tools_cppy     0.21.0
- tools          @PROJECT_VERSION@
- scripts        @PROJECT_VERSION@
- algorithms     0.5.2
- navigation     0.14.0
- echosounders_cppy 0.28.0
- echosounders   0.28.0
- pingprocessing_cppy 0.3.0
- pingprocessing @PROJECT_VERSION@
- gridding       @PROJECT_VERSION@


In [3]:
#create the output path
path_out = "../data"
path_in = "data_in/"
index_root = 'index'

os.makedirs(path_out,exist_ok=True)
os.makedirs(path_in,exist_ok=True)

## Explore and sort input data

In [4]:
#load all data
files_kongsbergall = []
for r,d,f in os.walk(path_in, followlinks=True):
    for file in f:
        file_path = r + '/' + file
        if file.endswith('.wcd') or file.endswith('.all'):
            files_kongsbergall.append(file_path)
        
files_kongsbergall.sort()
        
print(len(files_kongsbergall))

609


In [5]:
input_files = files_kongsbergall

In [6]:
index = index_functions.load_index_files(input_files, index_root = None)
index = index_functions.load_index_files(input_files, index_root = index_root, update_index = index)

In [7]:
#open and index files

fm = kongsbergall.KongsbergAllFileHandler_mapped(input_files, cached_index = index)

indexing files ⢀ 99% :02s<00m:00s] [Found: 9380783 datagrams in 609 files (51186MB)]                                                 
Initializing ping interface ⠄ 99% :09s<00m:00s] [Done]                                              


In [8]:
index_functions.update_index_files(fm.get_cached_file_index(), index_root=index_root)